In [15]:
# Ignore warnings
import os
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
import numpy as np

In this notebook, I wanted to do something I had in my mind for a while using tensorflow. In a CMPE course, we learned a unit called ALU, the Arithmetic Logic Unit.

ALU takes 2 numbers (in 8 bit format), and has many control inputs. For instance, control inputs can be add, shift, sub. Depending on which of these controls are high, ALU performns and arithmetic operation on the 8 bit inputs.

I wanted to see if I can train a layer that resembles an ALU with 4 variables: add, sub, mul, div.

Layer was not as succesful as I hoped.

In [16]:
class ALU(tf.Module):
    
    def __init__(self, random=True, **kwargs):
        super().__init__(**kwargs)

        if random:
            self.add = tf.Variable(tf.random.normal([1,], dtype = tf.dtypes.float64))
            self.sub = tf.Variable(tf.random.normal([1,], dtype = tf.dtypes.float64))
            self.mul = tf.Variable(tf.random.normal([1,], dtype = tf.dtypes.float64))
            self.div = tf.Variable(tf.random.normal([1,], dtype = tf.dtypes.float64))
        else:
            self.add = tf.Variable(1, dtype=tf.dtypes.float64)
            self.sub = tf.Variable(1, dtype=tf.dtypes.float64)
            self.mul= tf.Variable(1, dtype=tf.dtypes.float64)
            self.div = tf.Variable(1, dtype=tf.dtypes.float64)

    def __call__(self, x):
        return self.add * (x[0] + x[1]) + self.sub * (x[0] - x[1]) + self.mul * (x[0] * x[1]) + self.div * (x[0] / x[1])

    def __str__(self):
        return "ALU:\nadd: %.2f\nsub: %.2f\nmul: %.2f\ndiv: %.2f\n" % (self.add, self.sub, self.mul, self.div)

In [17]:
def loss(target_y, predicted_y):
    return tf.reduce_mean(tf.square(target_y - predicted_y))

In [18]:
def train(model, x, y_actual, learning_rate):

    with tf.GradientTape() as tape:
        y = model(x)
        current_loss = loss(y, y_actual)

    d_add, d_sub, d_mul, d_div = tape.gradient(current_loss, [model.add, model.sub, model.mul, model.div])

    model.add.assign_sub(learning_rate * d_add)
    model.sub.assign_sub(learning_rate * d_sub)
    model.mul.assign_sub(learning_rate * d_mul)
    model.div.assign_sub(learning_rate * d_div)

In [19]:
# Define a training loop
def training_loop(model, x, y_actual, epochs, learning_rate, print_every = 1):

    for epoch in range(epochs):
        # Update the model with the single giant batch
        train(model, x, y_actual, learning_rate)

        current_loss = loss(y_actual, model(x))

        if epoch%print_every == 0 or epoch == epochs-1:
            print("Epoch %2d: loss=%2.5f" % (epoch, current_loss))

In [24]:
model = ALU()

N_PAIRS = 10
x = np.zeros(shape=(2, N_PAIRS))
x1 = np.arange(3,3+N_PAIRS)
x2 = np.arange(5,5+N_PAIRS)
x[0] = x1
x[1] = - x2

x = tf.constant(x, dtype=tf.dtypes.float64)

y_actual = tf.constant(x1 + x2, dtype=tf.dtypes.float64)

training_loop(model, x, y_actual, 20, 0.000005, 5)

Epoch  0: loss=7993.71866
Epoch  5: loss=3102.20399
Epoch 10: loss=1211.00029
Epoch 15: loss=479.79213
Epoch 19: loss=234.37269


In [25]:
print(model)

ALU:
add: -0.52
sub: 1.85
mul: 0.02
div: -0.99



In [22]:
model(x).numpy()

array([-2.95230657, -1.59436145,  0.55833732,  3.48868127,  7.18716572,
       11.64808785, 16.86781858, 22.84393855, 29.5747728 , 37.05912494])

In [23]:
y_actual.numpy()

array([ 8., 10., 12., 14., 16., 18., 20., 22., 24., 26.])